In [ ]:
==================================================
ML LEARNING JOURNEY - DAY 12
==================================================
Week: 2 of 24
Day: 12 of 168
Date: October 26, 2025
Topic: Custom Dataset Creation & Image Annotation
Overall Progress: 7.1%

🎯 Today's Goals:
- Understand YOLO annotation format
- Learn coordinate conversion (YOLO ↔ XYXY)
- Compare annotation tools
- Understand dataset structure

In [5]:
# Imports
import os
from pathlib import Path
import yaml
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random
from collections import Counter

print("✅ Imports successful!")
print("✅ Ready to learn about dataset creation!\n")

✅ Imports successful!
✅ Ready to learn about dataset creation!



In [10]:
# ============================================================
# UNDERSTANDING YOLO ANNOTATION FORMAT
# ============================================================
print("\n" + "="*60)
print("EXERCISE 1.1: YOLO Annotation Format")
print("="*60)

print("""
YOLO DATASET STRUCTURE:
══════════════════════════════════════════════════════════
your_dataset/
├── data.yaml              # Dataset configuration file
├── train/
│   ├── images/           # Training images
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   └── labels/           # Training labels (same names as images)
│       ├── image1.txt
│       ├── image2.txt
│       └── ...
├── val/
│   ├── images/           # Validation images
│   └── labels/           # Validation labels
└── test/                 # Optional test set
    ├── images/
    └── labels/

══════════════════════════════════════════════════════════
YOLO LABEL FORMAT (in .txt files):
══════════════════════════════════════════════════════════
Each line in the .txt file represents ONE object:

<class_id> <x_center> <y_center> <width> <height>

Where:
- class_id: Integer (0, 1, 2, ...) representing the class
- x_center: Center X coordinate (normalized 0-1)
- y_center: Center Y coordinate (normalized 0-1)  
- width: Box width (normalized 0-1)
- height: Box height (normalized 0-1)

EXAMPLE - image1.txt:
0 0.5 0.5 0.3 0.4
1 0.7 0.3 0.2 0.25
0 0.2 0.8 0.15 0.2

Translation:
- Line 1: Class 0 (person) at center (0.5, 0.5), size 0.3×0.4
- Line 2: Class 1 (helmet) at center (0.7, 0.3), size 0.2×0.25
- Line 3: Class 0 (person) at center (0.2, 0.8), size 0.15×0.2

══════════════════════════════════════════════════════════
COORDINATE NORMALIZATION:
══════════════════════════════════════════════════════════
If image is 640×480 pixels and box is at:
- Top-left: (100, 50)
- Bottom-right: (300, 250)

Calculate YOLO format:
1. Get center:
   x_center_pixels = (100 + 300) / 2 = 200
   y_center_pixels = (50 + 250) / 2 = 150

2. Get dimensions:
   width_pixels = 300 - 100 = 200
   height_pixels = 250 - 50 = 200

3. Normalize (divide by image size):
   x_center = 200 / 640 = 0.3125
   y_center = 150 / 480 = 0.3125
   width = 200 / 640 = 0.3125
   height = 200 / 480 = 0.4167

4. YOLO label:
   0 0.3125 0.3125 0.3125 0.4167

══════════════════════════════════════════════════════════
DATA.YAML FORMAT:
══════════════════════════════════════════════════════════
# data.yaml
path: /path/to/your/dataset  # dataset root dir
train: train/images          # train images (relative to path)
val: val/images              # val images (relative to path)
test: test/images            # test images (optional)

# Classes
names:
  0: person
  1: helmet
  2: vest
  3: no-helmet
  4: no-vest

nc: 5  # number of classes
══════════════════════════════════════════════════════════
""")

print("✅ Exercise 1.1 Complete!")
print("\n📝 Key Takeaways:")
print("   - YOLO uses normalized coordinates (0-1)")
print("   - Format: <class> <x_center> <y_center> <width> <height>")
print("   - One .txt file per image (same name)")
print("   - data.yaml defines dataset structure")


EXERCISE 1.1: YOLO Annotation Format

YOLO DATASET STRUCTURE:
══════════════════════════════════════════════════════════
your_dataset/
├── data.yaml              # Dataset configuration file
├── train/
│   ├── images/           # Training images
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   └── labels/           # Training labels (same names as images)
│       ├── image1.txt
│       ├── image2.txt
│       └── ...
├── val/
│   ├── images/           # Validation images
│   └── labels/           # Validation labels
└── test/                 # Optional test set
    ├── images/
    └── labels/

══════════════════════════════════════════════════════════
YOLO LABEL FORMAT (in .txt files):
══════════════════════════════════════════════════════════
Each line in the .txt file represents ONE object:

<class_id> <x_center> <y_center> <width> <height>

Where:
- class_id: Integer (0, 1, 2, ...) representing the class
- x_center: Center X coordinate (normalized 0-1)
- y_center: C

In [12]:
# ============================================================
# COORDINATE CONVERSION FUNCTIONS
# ============================================================
print("\n" + "="*60)
print("EXERCISE 1.2: Coordinate Conversion")
print("="*60)

def xyxy_to_yolo(x1, y1, x2, y2, img_width, img_height):
    """
    Convert corner coordinates to YOLO format
    
    Args:
        x1, y1: Top-left corner
        x2, y2: Bottom-right corner
        img_width, img_height: Image dimensions
    
    Returns:
        x_center, y_center, width, height (all normalized 0-1)
    """
    # Calculate center
    x_center = (x1 + x2) / 2
    y_center = (y1 + y2) / 2
    
    # Calculate dimensions
    width = x2 - x1
    height = y2 - y1
    
    # Normalize
    x_center_norm = x_center / img_width
    y_center_norm = y_center / img_height
    width_norm = width / img_width
    height_norm = height / img_height
    
    return x_center_norm, y_center_norm, width_norm, height_norm


def yolo_to_xyxy(x_center, y_center, width, height, img_width, img_height):
    """
    Convert YOLO format to corner coordinates
    
    Args:
        x_center, y_center,
        width, height: YOLO format (normalized)
        img_width, img_height: Image dimensions
    
    Returns:
        x1, y1, x2, y2: Corner coordinates in pixels
    """
    # Denormalize
    x_center_pixels = x_center * img_width
    y_center_pixels = y_center * img_height
    width_pixels = width * img_width
    height_pixels = height * img_height
    
    # Calculate corners
    x1 = int(x_center_pixels - width_pixels / 2)
    y1 = int(y_center_pixels - height_pixels / 2)
    x2 = int(x_center_pixels + width_pixels / 2)
    y2 = int(y_center_pixels + height_pixels / 2)
    
    return x1, y1, x2, y2


# Test the conversions
print("\n🧪 Testing Coordinate Conversions:")
print("-" * 60)

# Test case: 640×480 image
img_w, img_h = 640, 480
test_box = (100, 50, 300, 250)  # x1, y1, x2, y2

print(f"\nOriginal box (pixels): {test_box}")
print(f"Image size: {img_w}×{img_h}")

# Convert to YOLO
yolo_coords = xyxy_to_yolo(*test_box, img_w, img_h)
print(f"\nYOLO format: {[f'{x:.4f}' for x in yolo_coords]}")

# Convert back
reconstructed = yolo_to_xyxy(*yolo_coords, img_w, img_h)
print(f"Reconstructed (pixels): {reconstructed}")

# Verify
if test_box == reconstructed:
    print("\n✅ Conversion successful! Perfect round-trip!")
else:
    print("\n⚠️  Small rounding differences (normal due to float conversion)")

print("\n✅ Exercise 1.2 Complete!")
print("\n📝 These functions are essential for:")
print("   - Creating labels from annotation tools")
print("   - Visualizing YOLO predictions")
print("   - Converting between formats")


EXERCISE 1.2: Coordinate Conversion

🧪 Testing Coordinate Conversions:
------------------------------------------------------------

Original box (pixels): (100, 50, 300, 250)
Image size: 640×480

YOLO format: ['0.3125', '0.3125', '0.3125', '0.4167']
Reconstructed (pixels): (100, 50, 300, 250)

✅ Conversion successful! Perfect round-trip!

✅ Exercise 1.2 Complete!

📝 These functions are essential for:
   - Creating labels from annotation tools
   - Visualizing YOLO predictions
   - Converting between formats


In [13]:
# ============================================================
# PART 2: IMAGE ANNOTATION TOOLS & PRACTICE
# ============================================================
print("\n" + "="*60)
print("PART 2: IMAGE ANNOTATION TOOLS")
print("="*60)
print("\nGoals:")
print("   • Install annotation tools")
print("   • Practice labeling images")
print("   • Export in YOLO format")

print("\n" + "="*60)
print("EXERCISE 2.1: Annotation Tools Comparison")
print("="*60)

print("""
ANNOTATION TOOL OPTIONS:
══════════════════════════════════════════════════════════
1. ROBOFLOW (Recommended for beginners)
   Website: https://roboflow.com
   
   Pros:
   ✓ Web-based (no installation)
   ✓ Auto-labeling with AI assistance
   ✓ Built-in augmentation
   ✓ Direct YOLO export
   ✓ Dataset versioning
   ✓ Team collaboration
   ✓ Free tier: 3 projects, 10k images
   
   Cons:
   ✗ Requires internet
   ✗ Free tier limited
   
   Best for: Quick start, team projects, cloud workflow

──────────────────────────────────────────────────────────
2. LABELIMG (Good for offline work)
   GitHub: https://github.com/HumanSignal/labelImg
   
   Pros:
   ✓ Free and open source
   ✓ Works offline
   ✓ Simple interface
   ✓ Direct YOLO format support
   ✓ Fast keyboard shortcuts
   
   Cons:
   ✗ Manual installation
   ✗ No auto-labeling
   ✗ Basic features only
   
   Best for: Offline work, simple projects, full control

──────────────────────────────────────────────────────────
3. CVAT (Computer Vision Annotation Tool)
   Website: https://www.cvat.ai
   
   Pros:
   ✓ Professional-grade
   ✓ Video annotation support
   ✓ AI-assisted labeling
   ✓ Many export formats
   
   Cons:
   ✗ Steeper learning curve
   ✗ Overkill for simple projects
   
   Best for: Large teams, video data, complex projects

──────────────────────────────────────────────────────────
4. MAKESENSE.AI
   Website: https://www.makesense.ai
   
   Pros:
   ✓ Web-based (no signup)
   ✓ Privacy-focused (local processing)
   ✓ Simple and fast
   ✓ YOLO export
   
   Cons:
   ✗ Limited features
   ✗ No cloud storage
   
   Best for: Quick annotation, privacy concerns

══════════════════════════════════════════════════════════
RECOMMENDATION FOR YOUR PROJECT:
══════════════════════════════════════════════════════════
For Safety Equipment Detection (Project 1):
→ Use ROBOFLOW

Why?
1. Auto-labeling speeds up annotation 10x
2. Built-in augmentation (flip, rotate, brightness)
3. Easy dataset versioning
4. Perfect for learning
5. Can export to YOLO format instantly

Backup option: LabelImg (if no internet access)
══════════════════════════════════════════════════════════
""")

print("✅ Exercise 2.1 Complete!")
print("\nNext: Setting up Roboflow...")


PART 2: IMAGE ANNOTATION TOOLS

Goals:
   • Install annotation tools
   • Practice labeling images
   • Export in YOLO format

EXERCISE 2.1: Annotation Tools Comparison

ANNOTATION TOOL OPTIONS:
══════════════════════════════════════════════════════════
1. ROBOFLOW (Recommended for beginners)
   Website: https://roboflow.com

   Pros:
   ✓ Web-based (no installation)
   ✓ Auto-labeling with AI assistance
   ✓ Built-in augmentation
   ✓ Direct YOLO export
   ✓ Dataset versioning
   ✓ Team collaboration
   ✓ Free tier: 3 projects, 10k images

   Cons:
   ✗ Requires internet
   ✗ Free tier limited

   Best for: Quick start, team projects, cloud workflow

──────────────────────────────────────────────────────────
2. LABELIMG (Good for offline work)
   GitHub: https://github.com/HumanSignal/labelImg

   Pros:
   ✓ Free and open source
   ✓ Works offline
   ✓ Simple interface
   ✓ Direct YOLO format support
   ✓ Fast keyboard shortcuts

   Cons:
   ✗ Manual installation
   ✗ No auto-labelin

In [14]:
# ============================================================
# ROBOFLOW SETUP & WORKFLOW
# ============================================================
print("\n" + "="*60)
print("EXERCISE 2.2: Roboflow Setup")
print("="*60)

print("""
ROBOFLOW SETUP - STEP BY STEP:
══════════════════════════════════════════════════════════
STEP 1: Create Account
──────────────────────────────────────────────────────────
1. Go to: https://roboflow.com
2. Click "Get Started Free"
3. Sign up with email or GitHub
4. Verify your email

STEP 2: Create Your First Project
──────────────────────────────────────────────────────────
1. Click "Create New Project"
2. Name: "Safety-Equipment-Detector"
3. Project Type: "Object Detection"
4. Annotation Group: Create new workspace
5. Click "Create Project"

STEP 3: Define Classes
──────────────────────────────────────────────────────────
Add these classes (in order):
1. person
2. helmet
3. vest
4. no-helmet
5. no-vest

Tips:
- Keep class names lowercase
- No spaces (use hyphens)
- Order matters for class IDs

STEP 4: Upload Images
──────────────────────────────────────────────────────────
1. Click "Upload" button
2. Select 10-20 construction site images
   (Download free images from Pexels/Unsplash if needed)
3. Wait for upload to complete
4. Click "Finish Uploading"

STEP 5: Annotate Images
──────────────────────────────────────────────────────────
1. Click on an image to open annotator
2. Select class from left sidebar
3. Draw bounding box (click and drag)
4. Press 'Enter' to confirm box
5. Repeat for all objects in image
6. Click "Save" when done
7. Move to next image

Keyboard Shortcuts:
- 'w' = Select class 1
- 'e' = Select class 2  
- 'Enter' = Confirm box
- 'd' = Next image
- 'a' = Previous image
- 'Esc' = Cancel current box

STEP 6: Use Auto-Label (AI Assist)
──────────────────────────────────────────────────────────
After annotating 5-10 images manually:
1. Click "Auto-Label" button
2. Select "Use Roboflow AI"
3. Review and correct auto-generated labels
4. Saves 50-70% of time!

STEP 7: Generate Dataset Version
──────────────────────────────────────────────────────────
1. Click "Generate" button
2. Choose preprocessing:
   ✓ Auto-Orient
   ✓ Resize to 640x640
3. Choose augmentations:
   ✓ Flip: Horizontal
   ✓ Brightness: ±15%
   ✓ Exposure: ±15%
4. Train/Val/Test Split: 70/20/10
5. Click "Generate"

STEP 8: Export to YOLOv8
──────────────────────────────────────────────────────────
1. Click "Export" button
2. Format: "YOLOv8"
3. Download ZIP file
4. Extract to your project folder

STEP 9: Verify Dataset Structure
──────────────────────────────────────────────────────────
Extracted folder should look like:

Safety-Equipment-Detector-1/
├── data.yaml
├── train/
│   ├── images/
│   └── labels/
├── valid/
│   ├── images/
│   └── labels/
└── test/
    ├── images/
    └── labels/

══════════════════════════════════════════════════════════
PRACTICE ASSIGNMENT:
══════════════════════════════════════════════════════════
Your task:
1. Sign up for Roboflow
2. Create "Safety-Equipment-Detector" project
3. Find 10 construction site images online
4. Annotate at least 5 images manually
5. Try auto-label on remaining 5
6. Generate dataset with augmentation
7. Export as YOLOv8 format

Time estimate: 45-60 minutes

WHERE TO GET SAMPLE IMAGES:
- Pexels.com (search "construction worker")
- Unsplash.com (search "construction site")
- Pixabay.com (search "hard hat")

All free for commercial use!
══════════════════════════════════════════════════════════
""")

print("✅ Exercise 2.2 Complete!")
print("\nAction Required:")
print("1. Go to Roboflow.com")
print("2. Create your project")
print("3. Annotate 10 images")
print("4. Come back when ready for Part 3!")


EXERCISE 2.2: Roboflow Setup

ROBOFLOW SETUP - STEP BY STEP:
══════════════════════════════════════════════════════════
STEP 1: Create Account
──────────────────────────────────────────────────────────
1. Go to: https://roboflow.com
2. Click "Get Started Free"
3. Sign up with email or GitHub
4. Verify your email

STEP 2: Create Your First Project
──────────────────────────────────────────────────────────
1. Click "Create New Project"
2. Name: "Safety-Equipment-Detector"
3. Project Type: "Object Detection"
4. Annotation Group: Create new workspace
5. Click "Create Project"

STEP 3: Define Classes
──────────────────────────────────────────────────────────
Add these classes (in order):
1. person
2. helmet
3. vest
4. no-helmet
5. no-vest

Tips:
- Keep class names lowercase
- No spaces (use hyphens)
- Order matters for class IDs

STEP 4: Upload Images
──────────────────────────────────────────────────────────
1. Click "Upload" button
2. Select 10-20 construction site images
   (Download fr

In [15]:
# ============================================================
# LABELIMG SETUP (ALTERNATIVE)
# ============================================================
print("\n" + "="*60)
print("EXERCISE 2.3: LabelImg Setup (Optional)")
print("="*60)

print("""
LABELIMG - OFFLINE ANNOTATION TOOL:
══════════════════════════════════════════════════════════
INSTALLATION:
──────────────────────────────────────────────────────────
Method 1: Using pip (Easiest)
$ pip install labelImg
$ labelImg

Method 2: Using conda
$ conda install -c conda-forge labelimg
$ labelImg

Method 3: From source (if above fail)
$ git clone https://github.com/HumanSignal/labelImg.git
$ cd labelImg
$ pip install -r requirements.txt
$ python labelImg.py

──────────────────────────────────────────────────────────
USAGE:
──────────────────────────────────────────────────────────
1. Launch LabelImg:
   $ labelImg

2. Open directory of images:
   File → Open Dir → Select your images folder

3. Change save directory:
   File → Change Save Dir → Select labels folder

4. Set output format to YOLO:
   Click "PascalVOC" button → Changes to "YOLO"

5. Create classes file:
   Create "classes.txt" in same folder with:
   person
   helmet
   vest
   no-helmet
   no-vest

6. Start annotating:
   - Press 'w' to start box
   - Click and drag
   - Select class
   - Press 'd' for next image

Keyboard Shortcuts:
- w = Create box
- d = Next image
- a = Previous image
- del = Delete selected box
- Ctrl+S = Save
- Ctrl+D = Copy current label

──────────────────────────────────────────────────────────
PROS OF LABELIMG:
✓ Works offline
✓ Fast with keyboard shortcuts
✓ Direct YOLO format
✓ Free and open source

CONS:
✗ No auto-labeling
✗ No augmentation
✗ Manual setup

══════════════════════════════════════════════════════════
WHEN TO USE LABELIMG:
- No internet connection
- Privacy concerns (sensitive data)
- Simple, quick annotations
- Full control over process

WHEN TO USE ROBOFLOW:
- Learning YOLO (easier workflow)
- Need auto-labeling (faster)
- Want augmentation built-in
- Team collaboration
══════════════════════════════════════════════════════════
""")

print("✅ Exercise 2.3 Complete!")
print("\nYou now know BOTH annotation tools!")


EXERCISE 2.3: LabelImg Setup (Optional)

LABELIMG - OFFLINE ANNOTATION TOOL:
══════════════════════════════════════════════════════════
INSTALLATION:
──────────────────────────────────────────────────────────
Method 1: Using pip (Easiest)
$ pip install labelImg
$ labelImg

Method 2: Using conda
$ conda install -c conda-forge labelimg
$ labelImg

Method 3: From source (if above fail)
$ git clone https://github.com/HumanSignal/labelImg.git
$ cd labelImg
$ pip install -r requirements.txt
$ python labelImg.py

──────────────────────────────────────────────────────────
USAGE:
──────────────────────────────────────────────────────────
1. Launch LabelImg:
   $ labelImg

2. Open directory of images:
   File → Open Dir → Select your images folder

3. Change save directory:
   File → Change Save Dir → Select labels folder

4. Set output format to YOLO:
   Click "PascalVOC" button → Changes to "YOLO"

5. Create classes file:
   Create "classes.txt" in same folder with:
   person
   helmet
   ves

In [16]:
# ============================================================
# PART 3: DATASET VALIDATION & QUALITY CHECKS
# ============================================================
print("\n" + "="*60)
print("PART 3: DATASET VALIDATION")
print("="*60)

print("""
DATASET VALIDATION CHECKLIST:
══════════════════════════════════════════════════════════

1. FOLDER STRUCTURE ✓
   - train/images/ exists
   - train/labels/ exists
   - valid/images/ exists
   - valid/labels/ exists
   - test/images/ exists (optional)
   - test/labels/ exists (optional)

2. FILE PAIRING ✓
   - Every image has a corresponding label file
   - Same filename (except extension)
   - image.jpg → image.txt

3. LABEL FORMAT ✓
   - Values are space-separated
   - Class ID is integer (0, 1, 2...)
   - Coordinates are floats (0.0 to 1.0)
   - 5 values per line: class x y w h

4. COORDINATE VALIDITY ✓
   - All values between 0 and 1
   - x_center ± width/2 stays in [0,1]
   - y_center ± height/2 stays in [0,1]

5. CLASS DISTRIBUTION ✓
   - Check if balanced (no class <10% of total)
   - Identify underrepresented classes
   - Plan to collect more data if needed

6. DATA.YAML ✓
   - Path is correct
   - Class names match annotations
   - Number of classes (nc) is correct

══════════════════════════════════════════════════════════
""")

print("✅ Dataset validation checklist reviewed!")


PART 3: DATASET VALIDATION

DATASET VALIDATION CHECKLIST:
══════════════════════════════════════════════════════════

1. FOLDER STRUCTURE ✓
   - train/images/ exists
   - train/labels/ exists
   - valid/images/ exists
   - valid/labels/ exists
   - test/images/ exists (optional)
   - test/labels/ exists (optional)

2. FILE PAIRING ✓
   - Every image has a corresponding label file
   - Same filename (except extension)
   - image.jpg → image.txt

3. LABEL FORMAT ✓
   - Values are space-separated
   - Class ID is integer (0, 1, 2...)
   - Coordinates are floats (0.0 to 1.0)
   - 5 values per line: class x y w h

4. COORDINATE VALIDITY ✓
   - All values between 0 and 1
   - x_center ± width/2 stays in [0,1]
   - y_center ± height/2 stays in [0,1]

5. CLASS DISTRIBUTION ✓
   - Check if balanced (no class <10% of total)
   - Identify underrepresented classes
   - Plan to collect more data if needed

6. DATA.YAML ✓
   - Path is correct
   - Class names match annotations
   - Number of classe

In [17]:
# ============================================================
# DATA AUGMENTATION EXPLAINED
# ============================================================
print("\n" + "="*60)
print("EXERCISE 3.1: Data Augmentation Concepts")
print("="*60)

print("""
WHY AUGMENTATION?
══════════════════════════════════════════════════════════
- Increases dataset size (22 images → 66 images)
- Improves model robustness
- Prevents overfitting
- Simulates real-world variations

COMMON AUGMENTATION TECHNIQUES:
══════════════════════════════════════════════════════════

1. FLIP (Horizontal/Vertical)
   - Mirror image left-right or up-down
   - Doubles dataset size instantly
   - Use: Horizontal flip for most objects

2. ROTATION
   - Rotate image by small angles (±15°)
   - Simulates camera angle variations
   - Careful: May not work for orientation-sensitive objects

3. BRIGHTNESS/CONTRAST
   - Adjust lighting conditions
   - Simulates different times of day
   - ±15-20% is typical

4. CROP & ZOOM
   - Random crops with padding
   - Simulates different distances
   - Forces model to learn partial objects

5. NOISE
   - Add Gaussian or salt-and-pepper noise
   - Simulates low-quality images
   - Improves robustness

6. MOSAIC (YOLO-specific)
   - Combines 4 images into one
   - Forces learning of multiple scales
   - Very effective for YOLO

ROBOFLOW AUGMENTATION:
══════════════════════════════════════════════════════════
Roboflow applies augmentation automatically:
- 1 source image → 3 augmented versions (2x flip, 1x brightness)
- 22 source images → 66 total images
- All augmentations preserve bounding boxes!

══════════════════════════════════════════════════════════
""")

print("✅ Exercise 3.1 Complete!")
print("\n📝 Augmentation = More data without collecting more images!")


EXERCISE 3.1: Data Augmentation Concepts

WHY AUGMENTATION?
══════════════════════════════════════════════════════════
- Increases dataset size (22 images → 66 images)
- Improves model robustness
- Prevents overfitting
- Simulates real-world variations

COMMON AUGMENTATION TECHNIQUES:
══════════════════════════════════════════════════════════

1. FLIP (Horizontal/Vertical)
   - Mirror image left-right or up-down
   - Doubles dataset size instantly
   - Use: Horizontal flip for most objects

2. ROTATION
   - Rotate image by small angles (±15°)
   - Simulates camera angle variations
   - Careful: May not work for orientation-sensitive objects

3. BRIGHTNESS/CONTRAST
   - Adjust lighting conditions
   - Simulates different times of day
   - ±15-20% is typical

4. CROP & ZOOM
   - Random crops with padding
   - Simulates different distances
   - Forces model to learn partial objects

5. NOISE
   - Add Gaussian or salt-and-pepper noise
   - Simulates low-quality images
   - Improves robust

In [19]:
# ============================================================
# DAY 12 COMPLETE: CUSTOM DATASET CREATION
# ============================================================
print("\n" + "="*60)
print("🎉 DAY 12 COMPLETE!")
print("="*60)

print("""
══════════════════════════════════════════════════════════
DAY 12 COMPLETE: CUSTOM DATASET CREATION
══════════════════════════════════════════════════════════
Date: October 26, 2025
Time Investment: ~3-4 hours
EXERCISES COMPLETED: 3/3

──────────────────────────────────────────────────────────
Part 1: YOLO Dataset Format (✅ Complete)
├─ Exercise 1.1: Understanding YOLO format
├─ Exercise 1.2: Coordinate conversion functions
└─ Learning: Label structure, normalized coordinates

Part 2: Image Annotation (✅ Complete)
├─ Exercise 2.1: Annotation tools comparison
├─ Exercise 2.2: Roboflow setup & annotation
├─ Exercise 2.3: LabelImg alternative
└─ Learning: Web-based vs offline workflows

Part 3: Dataset Validation (✅ Complete)
├─ Exercise 3.1: Data augmentation concepts
└─ Learning: Quality checks, class distribution

KEY LEARNINGS:
──────────────
🧠 Dataset Quality:
   • More data > perfect annotations
   • Auto-label saves 50-70% time
   • Class balance affects learning
   • Augmentation multiplies your data

🧠 YOLO Format:
   • Normalized coordinates (0-1)
   • Format: class x_center y_center width height
   • One .txt file per image
   • data.yaml defines structure

🧠 Production Skills:
   • Roboflow workflow
   • Dataset structure management
   • Quality validation process
   • Understanding augmentation

READY FOR:
──────────
✅ Day 13: Model training & optimization
✅ Understanding training metrics
✅ Hyperparameter tuning
✅ Model evaluation

Week 2 Progress: 71% (5/7 days)
Overall Progress: 7.1% (12/168 days)

STATUS: DAY 12 COMPLETE! 🎉
NEXT: Day 13 - Train & Optimize YOLO Model
""")

print("\n💪 CRUSHING Week 2!")
print("\n🚀 Tomorrow: Train my first YOLO model!")


🎉 DAY 12 COMPLETE!

══════════════════════════════════════════════════════════
DAY 12 COMPLETE: CUSTOM DATASET CREATION
══════════════════════════════════════════════════════════
Date: October 26, 2025
Time Investment: ~3-4 hours
EXERCISES COMPLETED: 3/3

──────────────────────────────────────────────────────────
Part 1: YOLO Dataset Format (✅ Complete)
├─ Exercise 1.1: Understanding YOLO format
├─ Exercise 1.2: Coordinate conversion functions
└─ Learning: Label structure, normalized coordinates

Part 2: Image Annotation (✅ Complete)
├─ Exercise 2.1: Annotation tools comparison
├─ Exercise 2.2: Roboflow setup & annotation
├─ Exercise 2.3: LabelImg alternative
└─ Learning: Web-based vs offline workflows

Part 3: Dataset Validation (✅ Complete)
├─ Exercise 3.1: Data augmentation concepts
└─ Learning: Quality checks, class distribution

KEY LEARNINGS:
──────────────
🧠 Dataset Quality:
   • More data > perfect annotations
   • Auto-label saves 50-70% time
   • Class balance affects learni